# Unlocking ComfyUI's Power: A Guide to the HTTP API in Jupyter

[ComfyUI](https://github.com/comfyanonymous/ComfyUI) is a powerful, modular interface for Stable Diffusion, allowing users to create complex AI image generation workflows with a node-based editor. [Jupyter Notebook](https://jupyter.org/), on the other hand, is a popular interactive environment for data analysis, visualization, and prototyping.

By integrating ComfyUI with Jupyter Notebook, you can leverage the flexibility of ComfyUI’s workflows directly within your Python scripts or data science pipelines. Let's focus on a simple approach using Basic HTTP API calls.

## The Use Case

Our goal is to build a high-level generative AI workflow that combines the power of an intelligent agent with the robust image generation capabilities of ComfyUI. The process unfolds in a few simple steps, all orchestrated within a Jupyter Notebook:

1. User Input: The workflow begins with a simple, high-level prompt entered directly into the notebook.
2. Agent-Powered Expansion: An OpenAI Assistant then takes this basic prompt and transforms it into a detailed, structured JSON Prompt Style Guide. This process enriches the initial idea with specific creative instructions, such as style, composition, and lighting.
3. Initiating Generation: This expanded JSON guide is automatically injected into a pre-defined ComfyUI workflow. A single API call to the ComfyUI server starts the image generation process.
4. Displaying the Result: Once the generation is complete, we make a second API call to fetch the resulting images. The images are then displayed directly within the Jupyter Notebook, completing our automated pipeline.


## Prepare a ComfyUI Workflow

- Create or load a workflow in ComfyUI.
- Save the workflow as a .json file from the "*File / Export (API)*" menu (e.g., `t2i-krea.json`).

## Get initial prompt from user

In [2]:
print("Please enter your prompt")
user_prompt = input()

Please enter your prompt
A cowboy riding a giant flying lizard


## Generate JSON Prompt Guide with an Assistant

In [3]:
import os
from openai import OpenAI
from dotenv import load_dotenv

load_dotenv()

client = OpenAI(api_key=os.environ.get("OPENAI_API_KEY"))

# Create a thread
thread = client.beta.threads.create()

# Send a message
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content=user_prompt
)

# Run the assistant
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id="asst_Uj0Qr0rG0bz8NVk1LWiS9UKv"
)

# Wait for completion and retrieve the response
import time
while run.status != "completed":
    time.sleep(1)
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)

# Get the response
messages = client.beta.threads.messages.list(thread_id=thread.id)
json_prompt = messages.data[0].content[0].text.value
print(json_prompt)


{
  "style_name": "Wild Frontier Fantasy",
  "inspiration": ["American Old West Art", "Dragon Illustrations", "Classic Westerns"],
  "scene": "Grand, expansive Wild West landscape with an open, cloud-filled sky",
  "subjects": [
    {
      "type": "cowboy",
      "description": "weathered, stern face, straw hat, just visible under the broad brim, wearing a duster coat and leather boots",
      "position": "center",
      "pose": "riding",
      "size": "medium",
      "expression": "resolved",
      "interaction": "guiding the flying lizard"
    },
    {
      "type": "flying lizard",
      "description": "giant lizard with leathery wings, colored in dusty shades of brown and green",
      "position": "background",
      "pose": "in flight",
      "size": "large",
      "expression": "ferocious",
      "interaction": "being ridden by the cowboy"
    }
  ],
  "style": "Semi-Realistic",
  "color_palette": {
    "primary": "#6d4c41",
    "secondary": "#74a089",
    "highlight": "#deb887"

## Trigger the Workflow from Jupyter Notebook

Use the requests library to send a POST request to the ComfyUI API:

In [4]:
import requests
import json

# ComfyUI server URL
comfy_url = "http://127.0.0.1:8188"
prompt_url = f"{comfy_url}/prompt"

# Load your workflow JSON
with open("t2i-krea.json", "r") as f:
    workflow = json.load(f)

# Replace the prompt
workflow["39:6"]["inputs"]["text"] = json_prompt
    
# Define the payload
payload = {
    "prompt": workflow,
    "client_id": "jupyter_notebook"
}

# Send the request
response = requests.post(prompt_url, json=payload)

# Get the prompt_id
prompt_id = response.json()['prompt_id']
print(prompt_id)


c14e6d4b-7667-42d3-a7b9-190acdcbc1e7


## Retrieve the generated images

ComfyUI processes the workflow asynchronously. 

To fetch the result, poll the /history endpoint:

In [5]:
import time
from IPython.display import Image, display    

# Wait for the workflow to complete
time.sleep(25)  # Adjust based on workflow complexity

# Fetch the latest result for our prompt
history_url = f"{comfy_url}/history/{prompt_id}"
history = requests.get(history_url).json()

# Navigate to the list of image outputs and display them
image_outputs = history[prompt_id]["outputs"]["9"]["images"]

for image in image_outputs:
    filename = image["filename"]
    image_url = f"{comfy_url}/view?filename={filename}"
    display(Image(url=image_url, width=200))
